# MTA Subway Hourly Ridership: 2020-2024

**Imports**

In [1]:
import pandas as pd
import numpy as np
import os

**Loading data**

In [ ]:
# Importing dataset
url = 'https://data.ny.gov/resource/wujg-7c2s.csv'
df = pd.read_csv(url)

# Display the first few rows of the dataset
print(df.head())

         transit_timestamp transit_mode station_complex_id  station_complex  \
0  2022-01-14T21:00:00.000       subway                 30  Prospect Av (R)   
1  2022-01-14T07:00:00.000       subway                204  Beach 67 St (A)   
2  2022-01-14T04:00:00.000       subway                 64        71 St (D)   
3  2022-01-14T06:00:00.000       subway                372  Longwood Av (6)   
4  2022-01-14T10:00:00.000       subway                 30  Prospect Av (R)   

    borough payment_method    fare_class_category  ridership  transfers  \
0  Brooklyn      metrocard  Metrocard - Fair Fare        1.0        0.0   
1    Queens      metrocard  Metrocard - Fair Fare        3.0        0.0   
2  Brooklyn      metrocard      Metrocard - Other        1.0        0.0   
3     Bronx      metrocard   Metrocard - Students       19.0        1.0   
4  Brooklyn      metrocard      Metrocard - Other        6.0        0.0   

    latitude  longitude                  georeference  
0  40.665413 -73.9

### Creating plots